# Rupaul's Drag Race Machine

In [122]:
import pandas as pd
from scipy.stats import rankdata, kendalltau

theData = pd.read_csv("dragrace.csv")
theData= theData.fillna(0)
#theData.iloc[1:10,:]

In [123]:
queens = theData.groupby('Name').max()
queens = queens.reset_index(drop=False)
#queens.iloc[1:10,:]

In [124]:
#queens.loc[queens.Place==1,:]

In [125]:
Xtrain = queens.loc[queens.Season<7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
Xtest = queens.loc[queens.Season==7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
ytrain = queens.loc[queens.Season<7,'Place']
ytest = queens.loc[queens.Season==7,'Place']
season8X = queens.loc[queens.Season==8,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
season8y = queens.loc[queens.Season==8,'Place']
season7 = queens.loc[queens.Season==7,['Name','Place']]
season7.columns = ['Name','Actual']
season7 = season7.reset_index(drop=True)
season8 = queens.loc[queens.Season==8,['Name','Place']]
season8.columns = ['Name','Actual']
season8 = season8.reset_index(drop=True)

In [126]:
def check_ranks(real,predicted):
    mymean = real.mean()
    x2 = real-mymean
    x2 = x2*x2
    sum1 = x2.sum()
    x2 = real-predicted
    x2 = x2*x2
    sum2 = x2.sum()
    r2 = 1 - (sum2/sum1)
    return r2

# Support Vector Machine Classifier

In [127]:
from sklearn.svm import SVC
model = SVC()
model.fit(Xtrain,ytrain)

yfit = model.predict(Xtest)
yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')
svc7 = pd.merge(season7,yfitpd).sort_values('Actual')
svc7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,5
9,Pearl,3,9
5,Kennedy Davenport,4,5
4,Katya,5,5
12,Trixie Mattel,6,9
7,Miss Fame,7,3
1,Jaidynn Diore Fierce,8,12
6,Max,9,9
3,Kandy Ho,10,4


In [128]:
kendalltau(svc7.Actual,svc7.Predicted)

KendalltauResult(correlation=0.22130457664129397, pvalue=0.27024792506861006)

## Season 8 Predictions

In [129]:
presentFit = model.predict(season8X)
presentFitPD = pd.DataFrame(presentFit)
presentFitPD['Name'] = season8['Name']
presentFitPD.columns = ['Predicted','Name']
presentFitPD['Predicted'] = rankdata(presentFitPD.Predicted,method='min')
svc8 = pd.merge(season8,presentFitPD).sort_values('Predicted')
svc8

,Name,Actual,Predicted
0,Acid Betty,0,1
5,Derrick Barry,0,2
10,Robbie Turner,0,2
11,Thorgy Thor,0,2
4,Dax ExclamationPoint,11,5
9,Naysha Lopez,12,5
6,Kim Chi,0,7
1,Bob the Drag Queen,0,8
2,Chi Chi DeVayne,0,8
3,Cynthia Lee Fontaine,0,8


# Gaussian Naive Bayes

In [130]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(Xtrain,ytrain)
nbTest = model.predict(Xtest)
nbTestDB = pd.DataFrame(nbTest)
nbTestDB['Name'] = season7.Name
nbTestDB.columns = ['Predicted','Name']
nbTestDB = nbTestDB.loc[:,['Name','Predicted']]
nbTestDB['Predicted'] = rankdata(nbTestDB.Predicted,method='min')

nb7 = pd.merge(season7,nbTestDB).sort_values('Actual')
nb7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,8
5,Kennedy Davenport,4,1
4,Katya,5,1
12,Trixie Mattel,6,8
7,Miss Fame,7,4
1,Jaidynn Diore Fierce,8,6
6,Max,9,8
3,Kandy Ho,10,4


In [131]:
kendalltau(nb7.Actual,nb7.Predicted)

KendalltauResult(correlation=0.48108780653777095, pvalue=0.016544547824374332)

## Season 8 Predictions

In [121]:
nb8 = model.predict(season8X)
nb8DB = pd.DataFrame(nb8)
nb8DB['Name'] = season8.Name
nb8DB.columns = ['Predicted','Name']
nb8DB['Predicted'] = rankdata(nb8DB.Predicted,method='min')
nb8 = pd.merge(season8,nb8DB).sort_values('Predicted')
nb8

,Name,Actual,Predicted
3,Cynthia Lee Fontaine,0,1
9,Naysha Lopez,12,1
0,Acid Betty,0,3
2,Chi Chi DeVayne,0,3
5,Derrick Barry,0,3
6,Kim Chi,0,3
8,Naomi Smalls,0,3
10,Robbie Turner,0,3
11,Thorgy Thor,0,3
1,Bob the Drag Queen,0,10


# Random Forest Classifier

In [132]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(Xtrain,ytrain)
yfit = model.predict(Xtest)

yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')

## Season 7 Predictions

In [133]:
rfClass = pd.merge(season7,yfitpd).sort_values('Actual')
rfClass

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,3
5,Kennedy Davenport,4,4
4,Katya,5,4
12,Trixie Mattel,6,6
7,Miss Fame,7,10
1,Jaidynn Diore Fierce,8,6
6,Max,9,1
3,Kandy Ho,10,6


In [135]:
kendalltau(rfClass.Actual,rfClass.Predicted)

KendalltauResult(correlation=0.64061851133006142, pvalue=0.0014157354382150771)

## Season 8 Predictions

In [96]:
rf8Fit = model.predict(season8X)
rf8FitPD = pd.DataFrame(rf8Fit)
rf8FitPD['Name'] = season8.Name
rf8FitPD.columns = ['Predicted','Name']
rf8FitPD = rf8FitPD.loc[:,['Name','Predicted']]
rf8FitPD['Predicted'] = rankdata(rf8FitPD.Predicted,method='min')

In [97]:
rfClass8 = pd.merge(season8,rf8FitPD).sort_values('Predicted')
rfClass8

,Name,Actual,Predicted
2,Chi Chi DeVayne,0,1
6,Kim Chi,0,1
10,Robbie Turner,0,1
0,Acid Betty,0,4
5,Derrick Barry,0,5
11,Thorgy Thor,0,5
8,Naomi Smalls,0,7
1,Bob the Drag Queen,0,8
4,Dax ExclamationPoint,11,8
7,Laila McQueen,11,8


# Random Forest Regressor

In [136]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(200,random_state=24601)
forest.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=24601,
           verbose=0, warm_start=False)

## Season 7 Predictions

In [137]:
forestFit = forest.predict(Xtest)
forestFitDB = pd.DataFrame(forestFit.round())
forestFitDB['Name'] = season7.Name
forestFitDB.columns = ['Predicted','Name']
forestFitDB['Predicted'] = rankdata(forestFitDB.Predicted,method='min')
rfReg7 = pd.merge(season7,forestFitDB).sort_values('Actual')
rfReg7

,Name,Actual,Predicted
13,Voilet Chachki,1,2
0,Ginger Minj,2,1
9,Pearl,3,6
5,Kennedy Davenport,4,2
4,Katya,5,2
12,Trixie Mattel,6,8
7,Miss Fame,7,8
1,Jaidynn Diore Fierce,8,6
6,Max,9,5
3,Kandy Ho,10,8


In [104]:
kendalltau(rfReg7.Actual,rfReg7.Predicted)

KendalltauResult(correlation=0.73379938570534309, pvalue=0.00025655048033250812)

## Season 8 Predictions

In [105]:
forest8 = forest.predict(season8X)
forest8DB = pd.DataFrame(forest8.round())
forest8DB['Name'] = season8.Name
forest8DB.columns = ['Predicted','Name']
forest8DB['Predicted'] = rankdata(forest8DB.Predicted,method='min')
pd.merge(season8,forest8DB).sort_values('Predicted')

,Name,Actual,Predicted
2,Chi Chi DeVayne,0,1
6,Kim Chi,0,1
0,Acid Betty,0,3
10,Robbie Turner,0,3
5,Derrick Barry,0,5
7,Laila McQueen,11,5
8,Naomi Smalls,0,5
11,Thorgy Thor,0,5
1,Bob the Drag Queen,0,9
4,Dax ExclamationPoint,11,9
